In [ ]:
import os
import pandas as pd
import numpy as np
import hds
from plt_rcs import *

In [ ]:
df = pd.read_csv('https://bit.ly/PimaIndiansDiabetes')

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.isna().sum()

In [ ]:
df.describe().round(3)

In [ ]:
df.columns

In [ ]:
cols = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']

In [ ]:
df[cols].apply(func=lambda x: x.eq(0).sum())

## 데이터 전처리

In [ ]:
df[cols] = df[cols].replace(0, np.nan)
df[cols].isna().sum()

In [ ]:
df.columns

In [ ]:
df = df.rename(columns={'DiabetesPedigreeFunction': 'Pedigree'})

In [ ]:
df.head()

## 회귀 대체

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [ ]:
imputer = IterativeImputer()

In [ ]:
df_imp = pd.DataFrame(data=imputer.fit_transform(X=df), columns=df.columns)

In [ ]:
df_imp.head()

In [ ]:
df_imp.isna().sum()

In [ ]:
df_imp[cols] = df_imp[cols].round(1)

In [ ]:
df_imp.head()

In [ ]:
df_imp.describe().round(3)

In [ ]:
df_imp.loc[df['Insulin'].isna(), 'Insulin'].sort_values()

In [ ]:
df_imp['Insulin'] = df_imp['Insulin'].clip(lower=16)

In [ ]:
df_imp.describe().round(3)

In [ ]:
plt.rc(group='figure', figsize=(4, 4))

In [ ]:
sns.histplot(data=df, x='Insulin', binrange=(0, 850), binwidth=50, fc='0.8')
plt.show()

In [ ]:
sns.kdeplot(
    data=df, x='Insulin',
    color='0.8', fill=True
)
plt.axvline(x=df_imp['Insulin'].mean())
plt.axvline(x=df_imp['Insulin'].median(), color='red', linestyle='--')
plt.show()

In [ ]:
hds.plot.corr_heatmap(data=df)

In [ ]:
var_names = df.columns.to_list()[:-1]

In [ ]:
for var in var_names:
    if var == 'Insulin':
        continue
    hds.plot.regline(data=df_imp, x=var, y='Insulin')
    plt.show()

In [ ]:
hds.plot.box_group(data=df_imp, x='Outcome', y='Insulin', palette=['skyblue', 'orange'])

In [ ]:
df_imp.columns

In [ ]:
cond1 = df_imp['SkinThickness'].lt(80)
cond2 = df_imp['BMI'].lt(60)
df_imp = df_imp.loc[cond1 & cond2, :].drop(columns='Outcome')
df_imp = df_imp.reset_index(drop=True)
df_imp.shape

In [ ]:
yvar = 'Insulin'
X = df_imp.drop(columns=yvar)
y = df_imp[yvar].copy()
display(X)
display(y)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
X_train.shape

In [ ]:
X_valid.shape

In [ ]:
y_train.mean()

In [ ]:
y_valid.mean()

In [ ]:
os.getcwd()

In [ ]:
os.chdir('../../data')

In [ ]:
objs = {
    'X_train': X_train,
    'X_valid': X_valid,
    'y_train': y_train,
    'y_valid': y_valid
}

In [ ]:
pd.to_pickle(obj=objs, filepath_or_buffer='Diabetes.pkl')